<a href="https://colab.research.google.com/github/nitishast/TF/blob/master/Image_Classifier_Using_Sequential_API_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import pandas as pd
import matplotlib as mp
import tensorflow as tf
from tensorflow import keras

In [30]:
keras.__version__

'2.2.4-tf'

### Loading the MNIST Dataset

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

### In scikit learn same image is represented as 1D array of 784 but in keras each is 28x28 array

In [0]:
(X_train_full,y_train_full),(X_test,y_test) = fashion_mnist.load_data()

In [33]:
print (X_train_full.shape)
print (X_train_full.dtype)

(60000, 28, 28)
uint8


### Since we will optimise the network by using Gradient descent, we must scale the input feature to 0-1. Can be done by dividing by max feature 255

In [0]:
# Creating a Validation set 
X_valid, X_train = X_train_full[:5000]/255.0 , X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000]/255.0 , y_train_full[5000:]

In [0]:
class_names = ["T-shirt/top" , "Trouser" , " Pullover", "Dress" , "Coat" , "Sandal" , "Shirt" , "Sneaker" , "Bag" , "Ankle boot"]

In [42]:
class_names[y_train[0]]

'Coat'

## Building a sequential model

In [0]:
from keras.models import Sequential
from keras.layers import Flatten,Dense

In [0]:
model = keras.models.Sequential()
#Input Layer
model.add(keras.layers.Flatten(input_shape=[28,28]))
#Fully connected layers -Dense
model.add(keras.layers.Dense(300,activation="relu"))
model.add(keras.layers.Dense(100,activation="relu"))

#Output layer
model.add(keras.layers.Dense(10,activation="softmax"))

